# Pragmatic Asset Allocation - Data Exploration

This notebook explores the asset data quality, statistical properties, and relationships for the Pragmatic Asset Allocation Model.

## Objectives:
- Assess data completeness and quality
- Analyze return distributions and correlations
- Evaluate liquidity and trading characteristics
- Check for survivorship bias and data issues

In [ ]:
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Load configuration
with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded successfully")
print(f"Strategy: {config['strategy']['name']}")
print(f"Backtest period: {config['backtest']['start_date']} to {config['backtest']['end_date']}")

## 1. Load and Validate Data

In [ ]:
# Load data from data acquisition
try:
    from data_acquisition import PragmaticAssetAllocationData
    
    data_acq = PragmaticAssetAllocationData()
    
    # Try to load cached data first
    all_data = data_acq.load_cached_data()
    
    if not all_data:
        print("No cached data found. Run data acquisition first.")
        print("Use: python ../main.py --mode data")
        all_data = {}
    else:
        print("Cached data loaded successfully")
        
except ImportError:
    print("Could not import data acquisition module")
    all_data = {}

In [ ]:
# Data quality assessment
if all_data:
    print("=== DATA QUALITY ASSESSMENT ===\n")
    
    for data_type, df in all_data.items():
        if isinstance(df, pd.DataFrame):
            print(f"{data_type.upper()}:")
            print(f"  Shape: {df.shape}")
            print(f"  Date range: {df.index.min()} to {df.index.max()}")
            print(f"  Missing values: {df.isnull().sum().sum()}")
            print(f"  Completeness: {(1 - df.isnull().sum().sum() / df.size):.1%}")
            
            # Check for data quality issues
            if df.isnull().sum().sum() > 0:
                print(f"  ⚠️  Missing data detected")
            else:
                print(f"  ✓ Complete data")
            print()
        else:
            print(f"{data_type}: Not a DataFrame")
else:
    print("No data available for analysis")

## 2. Risky Assets Analysis

In [ ]:
# Analyze risky assets
if 'risky_assets' in all_data:
    risky_data = all_data['risky_assets']
    print("=== RISKY ASSETS ANALYSIS ===\n")
    
    # Basic statistics
    risky_assets = config['assets']['risky']
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    for i, asset in enumerate(risky_assets):
        ticker = asset['ticker']
        if ticker in risky_data.columns.levels[0]:
            prices = risky_data[ticker]['Adj Close']
            returns = prices.pct_change().dropna()
            
            # Price chart
            ax = axes[i//3, i%3]
            prices.plot(ax=ax, linewidth=1)
            ax.set_title(f'{asset["name"]} ({ticker})')
            ax.set_xlabel('Date')
            ax.set_ylabel('Price ($)')
            ax.grid(True, alpha=0.3)
            
            # Print statistics
            print(f"{asset['name']} ({ticker}):")
            print(f"  Start Price: ${prices.iloc[0]:.2f}")
            print(f"  End Price: ${prices.iloc[-1]:.2f}")
            print(f"  Total Return: {((prices.iloc[-1]/prices.iloc[0])-1):.1%}")
            print(f"  Annual Volatility: {returns.std() * np.sqrt(252):.1%}")
            print(f"  Sharpe Ratio: {(returns.mean()/returns.std()) * np.sqrt(252):.2f}")
            print(f"  Max Drawdown: {((prices/prices.expanding().max()-1).min()):.1%}")
            print()
    
    plt.tight_layout()
    plt.show()
else:
    print("Risky assets data not available")

In [ ]:
# Return distribution analysis
if 'risky_assets' in all_data:
    risky_data = all_data['risky_assets']
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    for i, asset in enumerate(config['assets']['risky']):
        ticker = asset['ticker']
        if ticker in risky_data.columns.levels[0]:
            prices = risky_data[ticker]['Adj Close']
            returns = prices.pct_change().dropna()
            
            # Returns distribution
            ax = axes[i//3, i%3]
            ax.hist(returns, bins=50, alpha=0.7, density=True, edgecolor='black')
            ax.axvline(returns.mean(), color='red', linestyle='--', 
                      label=f'Mean: {returns.mean():.2%}')
            ax.axvline(returns.median(), color='blue', linestyle='--',
                      label=f'Median: {returns.median():.2%}')
            ax.set_title(f'{asset["name"]} Return Distribution')
            ax.set_xlabel('Daily Return')
            ax.set_ylabel('Density')
            ax.legend()
            ax.grid(True, alpha=0.3)
            
            # Add statistics text
            stats_text = f'Skew: {returns.skew():.2f}\nKurt: {returns.kurtosis():.2f}'
            ax.text(0.05, 0.95, stats_text, transform=ax.transAxes, 
                   verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics table
    print("\n=== RETURN DISTRIBUTION SUMMARY ===\n")
    summary_stats = []
    
    for asset in config['assets']['risky']:
        ticker = asset['ticker']
        if ticker in risky_data.columns.levels[0]:
            returns = risky_data[ticker]['Adj Close'].pct_change().dropna()
            
            stats = {
                'Asset': asset['name'],
                'Mean': returns.mean(),
                'Std': returns.std(),
                'Skewness': returns.skew(),
                'Kurtosis': returns.kurtosis(),
                'Min': returns.min(),
                'Max': returns.max(),
                '5% VaR': returns.quantile(0.05),
                '95% VaR': returns.quantile(0.95)
            }
            summary_stats.append(stats)
    
    if summary_stats:
        summary_df = pd.DataFrame(summary_stats)
        # Format as percentages
        pct_cols = ['Mean', 'Std', 'Min', 'Max', '5% VaR', '95% VaR']
        summary_df[pct_cols] = summary_df[pct_cols].apply(lambda x: x.map('{:.2%}'.format))
        summary_df[['Skewness', 'Kurtosis']] = summary_df[['Skewness', 'Kurtosis']].apply(lambda x: x.map('{:.2f}'.format))
        
        print(summary_df.to_string(index=False))

## 3. Hedging Assets Analysis

In [ ]:
# Analyze hedging assets
if 'hedging_assets' in all_data:
    hedging_data = all_data['hedging_assets']
    print("=== HEDGING ASSETS ANALYSIS ===\n")
    
    # Basic statistics
    hedging_assets = config['assets']['hedging']
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    for i, asset in enumerate(hedging_assets):
        ticker = asset['ticker']
        if ticker in hedging_data.columns.levels[0]:
            prices = hedging_data[ticker]['Adj Close']
            returns = prices.pct_change().dropna()
            
            # Price chart
            ax = axes[i]
            prices.plot(ax=ax, linewidth=1)
            ax.set_title(f'{asset["name"]} ({ticker})')
            ax.set_xlabel('Date')
            ax.set_ylabel('Price ($)')
            ax.grid(True, alpha=0.3)
            
            # Print statistics
            print(f"{asset['name']} ({ticker}):")
            print(f"  Start Price: ${prices.iloc[0]:.2f}")
            print(f"  End Price: ${prices.iloc[-1]:.2f}")
            print(f"  Total Return: {((prices.iloc[-1]/prices.iloc[0])-1):.1%}")
            print(f"  Annual Volatility: {returns.std() * np.sqrt(252):.1%}")
            print(f"  Sharpe Ratio: {(returns.mean()/returns.std()) * np.sqrt(252):.2f}")
            print(f"  Max Drawdown: {((prices/prices.expanding().max()-1).min()):.1%}")
            print()
    
    plt.tight_layout()
    plt.show()
else:
    print("Hedging assets data not available")

## 4. Correlation Analysis

In [ ]:
# Correlation analysis between all assets
if 'risky_assets' in all_data and 'hedging_assets' in all_data:
    print("=== CORRELATION ANALYSIS ===\n")
    
    # Combine all asset returns
    all_returns = pd.DataFrame()
    
    # Risky assets
    for asset in config['assets']['risky']:
        ticker = asset['ticker']
        if ticker in all_data['risky_assets'].columns.levels[0]:
            returns = all_data['risky_assets'][ticker]['Adj Close'].pct_change()
            all_returns[f'{ticker}'] = returns
    
    # Hedging assets
    for asset in config['assets']['hedging']:
        ticker = asset['ticker']
        if ticker in all_data['hedging_assets'].columns.levels[0]:
            returns = all_data['hedging_assets'][ticker]['Adj Close'].pct_change()
            all_returns[f'{ticker}'] = returns
    
    # Calculate correlation matrix
    corr_matrix = all_returns.corr()
    
    # Plot correlation heatmap
    plt.figure(figsize=(10, 8))
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', 
                center=0, square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
    plt.title('Asset Return Correlations')
    plt.tight_layout()
    plt.show()
    
    # Print correlation insights
    print("\nCorrelation Insights:")
    print(f"Average correlation between risky assets: {corr_matrix.loc[['QQQ', 'URTH', 'EEM'], ['QQQ', 'URTH', 'EEM']].mean().mean():.2f}")
    print(f"Average correlation between hedging assets: {corr_matrix.loc[['IEF', 'GLD'], ['IEF', 'GLD']].mean().mean():.2f}")
    
    # Risky vs Hedging correlations
    risky_hedge_corr = corr_matrix.loc[['QQQ', 'URTH', 'EEM'], ['IEF', 'GLD']].mean().mean()
    print(f"Average correlation between risky and hedging assets: {risky_hedge_corr:.2f}")
    
    if risky_hedge_corr < -0.2:
        print("✅ Good diversification: Negative correlation between risky and hedging assets")
    elif risky_hedge_corr > 0.5:
        print("⚠️ Poor diversification: High correlation between risky and hedging assets")
    else:
        print("✓ Moderate diversification between asset classes")
    
    # Rolling correlations (last 5 years if available)
    if len(all_returns) > 252*5:
        print("\n=== ROLLING CORRELATIONS (5-year windows) ===")
        rolling_corr = all_returns.rolling(window=252*5).corr()
        
        # Plot rolling correlations for key pairs
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # QQQ vs IEF
        if ('QQQ' in rolling_corr.columns.levels[0] and 
            'IEF' in rolling_corr.columns.levels[0]):
            qqq_ief_corr = rolling_corr.loc[pd.IndexSlice[:, 'QQQ'], 'IEF'].droplevel(0)
            qqq_ief_corr.plot(ax=axes[0,0], title='QQQ vs IEF Rolling Correlation')
            axes[0,0].grid(True, alpha=0.3)
        
        # QQQ vs GLD
        if ('QQQ' in rolling_corr.columns.levels[0] and 
            'GLD' in rolling_corr.columns.levels[0]):
            qqq_gld_corr = rolling_corr.loc[pd.IndexSlice[:, 'QQQ'], 'GLD'].droplevel(0)
            qqq_gld_corr.plot(ax=axes[0,1], title='QQQ vs GLD Rolling Correlation')
            axes[0,1].grid(True, alpha=0.3)
        
        # URTH vs IEF
        if ('URTH' in rolling_corr.columns.levels[0] and 
            'IEF' in rolling_corr.columns.levels[0]):
            urth_ief_corr = rolling_corr.loc[pd.IndexSlice[:, 'URTH'], 'IEF'].droplevel(0)
            urth_ief_corr.plot(ax=axes[1,0], title='URTH vs IEF Rolling Correlation')
            axes[1,0].grid(True, alpha=0.3)
        
        # EEM vs GLD
        if ('EEM' in rolling_corr.columns.levels[0] and 
            'GLD' in rolling_corr.columns.levels[0]):
            eem_gld_corr = rolling_corr.loc[pd.IndexSlice[:, 'EEM'], 'GLD'].droplevel(0)
            eem_gld_corr.plot(ax=axes[1,1], title='EEM vs GLD Rolling Correlation')
            axes[1,1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

## 5. Macroeconomic Data Analysis

In [ ]:
# Analyze macroeconomic indicators
if 'macroeconomic' in all_data:
    macro_data = all_data['macroeconomic']
    print("=== MACROECONOMIC ANALYSIS ===\n")
    
    # Yield curve analysis
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Treasury yields over time
    if '3M_Treasury_Yield' in macro_data.columns:
        macro_data['3M_Treasury_Yield'].plot(ax=axes[0,0], label='3M Yield', linewidth=1)
    if '10Y_Treasury_Yield' in macro_data.columns:
        macro_data['10Y_Treasury_Yield'].plot(ax=axes[0,0], label='10Y Yield', linewidth=1)
    axes[0,0].set_title('Treasury Yields Over Time')
    axes[0,0].set_ylabel('Yield (%)')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # Yield curve spread
    if 'Yield_Curve_Spread' in macro_data.columns:
        macro_data['Yield_Curve_Spread'].plot(ax=axes[0,1], linewidth=1, color='purple')
        axes[0,1].axhline(y=0, color='red', linestyle='--', alpha=0.7, label='Inversion Line')
        axes[0,1].set_title('Yield Curve Spread (10Y - 3M)')
        axes[0,1].set_ylabel('Spread (bps)')
        axes[0,1].legend()
        axes[0,1].grid(True, alpha=0.3)
        
        # Highlight inversion periods
        inversion_periods = macro_data[macro_data['Yield_Curve_Spread'] < 0]
        if not inversion_periods.empty:
            axes[0,1].fill_between(inversion_periods.index, inversion_periods['Yield_Curve_Spread'], 0, 
                                  color='red', alpha=0.3, label='Inverted')
    
    # Yield curve inversion frequency
    if 'Yield_Curve_Inverted' in macro_data.columns:
        inversion_pct = macro_data['Yield_Curve_Inverted'].mean()
        print(f"Yield curve inversion frequency: {inversion_pct:.1%}")
        
        # Plot inversion periods
        macro_data['Yield_Curve_Inverted'].astype(int).plot(ax=axes[1,0], linewidth=1, color='red')
        axes[1,0].set_title('Yield Curve Inversion Periods')
        axes[1,0].set_ylabel('Inverted (1=Yes, 0=No)')
        axes[1,0].grid(True, alpha=0.3)
        
        # Add recession overlays (simplified)
        recession_periods = [
            ('1937-05-01', '1938-06-01'),
            ('1940-10-01', '1941-03-01'),
            ('1945-02-01', '1945-10-01'),
            ('1948-11-01', '1949-10-01'),
            ('1953-07-01', '1954-05-01'),
            ('1957-08-01', '1958-04-01'),
            ('1960-04-01', '1961-02-01'),
            ('1969-12-01', '1970-11-01'),
            ('1973-11-01', '1975-03-01'),
            ('1980-01-01', '1980-07-01'),
            ('1981-07-01', '1982-11-01'),
            ('1990-07-01', '1991-03-01'),
            ('2001-03-01', '2001-11-01'),
            ('2007-12-01', '2009-06-01'),
            ('2020-02-01', '2020-04-01')
        ]
        
        for start, end in recession_periods:
            try:
                axes[1,0].axvspan(pd.to_datetime(start), pd.to_datetime(end), 
                                color='gray', alpha=0.3, label='Recession' if start == '1937-05-01' else "")
            except:
                pass
        
        axes[1,0].legend()
    
    # Spread distribution
    if 'Yield_Curve_Spread' in macro_data.columns:
        spread_data = macro_data['Yield_Curve_Spread'].dropna()
        axes[1,1].hist(spread_data, bins=50, alpha=0.7, density=True, edgecolor='black')
        axes[1,1].axvline(spread_data.mean(), color='red', linestyle='--', 
                         label=f'Mean: {spread_data.mean():.1f} bps')
        axes[1,1].axvline(0, color='black', linestyle='-', alpha=0.7, label='Inversion')
        axes[1,1].set_title('Yield Curve Spread Distribution')
        axes[1,1].set_xlabel('Spread (bps)')
        axes[1,1].set_ylabel('Density')
        axes[1,1].legend()
        axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print macroeconomic statistics
    print("\nMacroeconomic Statistics:")
    if 'Yield_Curve_Spread' in macro_data.columns:
        spread = macro_data['Yield_Curve_Spread'].dropna()
        print(f"Average yield curve spread: {spread.mean():.1f} bps")
        print(f"Spread volatility: {spread.std():.1f} bps")
        print(f"Minimum spread: {spread.min():.1f} bps")
        print(f"Maximum spread: {spread.max():.1f} bps")
        
        # Inversion analysis
        inversions = spread < 0
        print(f"Total inversion days: {inversions.sum()}")
        print(f"Inversion percentage: {inversions.mean():.1%}")
        
        if inversions.sum() > 0:
            print(f"Average inversion magnitude: {spread[inversions].mean():.1f} bps")
            print(f"Worst inversion: {spread.min():.1f} bps")
else:
    print("Macroeconomic data not available")

## 6. Data Quality Summary

In [ ]:
# Comprehensive data quality assessment
print("=== DATA QUALITY SUMMARY ===\n")

quality_checks = {
    'Data Completeness': [],
    'Survivorship Bias': [],
    'Liquidity Assessment': [],
    'Statistical Properties': []
}

# Data completeness
if all_data:
    total_rows = 0
    total_missing = 0
    
    for data_type, df in all_data.items():
        if isinstance(df, pd.DataFrame):
            total_rows += df.shape[0] * df.shape[1]
            total_missing += df.isnull().sum().sum()
    
    completeness = (1 - total_missing / total_rows) if total_rows > 0 else 0
    quality_checks['Data Completeness'].append(f"Overall completeness: {completeness:.1%}")
    
    if completeness > 0.95:
        quality_checks['Data Completeness'].append("✅ Excellent data completeness")
    elif completeness > 0.90:
        quality_checks['Data Completeness'].append("✓ Good data completeness")
    else:
        quality_checks['Data Completeness'].append("⚠️ Data completeness concerns")

# Survivorship bias assessment
start_date = pd.to_datetime(config['backtest']['start_date'])
end_date = pd.to_datetime(config['backtest']['end_date'])
expected_days = (end_date - start_date).days

if 'risky_assets' in all_data:
    risky_data = all_data['risky_assets']
    actual_days = len(risky_data)
    coverage = actual_days / expected_days if expected_days > 0 else 0
    
    quality_checks['Survivorship Bias'].append(f"Data coverage: {coverage:.1%} of expected period")
    
    if coverage > 0.8:
        quality_checks['Survivorship Bias'].append("✅ Good historical coverage")
    elif coverage > 0.6:
        quality_checks['Survivorship Bias'].append("✓ Acceptable historical coverage")
    else:
        quality_checks['Survivorship Bias'].append("⚠️ Limited historical coverage - survivorship bias possible")

# Liquidity assessment (simplified)
if 'risky_assets' in all_data:
    risky_data = all_data['risky_assets']
    
    # Check for ETF availability dates
    etf_start_dates = {}
    for asset in config['assets']['risky'] + config['assets']['hedging']:
        ticker = asset['ticker']
        if ticker in risky_data.columns.levels[0]:
            first_valid = risky_data[ticker]['Adj Close'].first_valid_index()
            etf_start_dates[ticker] = first_valid
    
    if etf_start_dates:
        earliest_etf = min(etf_start_dates.values())
        latest_etf = max(etf_start_dates.values())
        
        quality_checks['Liquidity Assessment'].append(f"ETF availability range: {earliest_etf.date()} to {latest_etf.date()}")
        
        # Check if we have data before ETF inception (proxy quality)
        if earliest_etf > start_date:
            quality_checks['Liquidity Assessment'].append("⚠️ ETF data starts after backtest begin - using proxies")
        else:
            quality_checks['Liquidity Assessment'].append("✅ ETF data covers full backtest period")

# Statistical properties check
if 'risky_assets' in all_data:
    risky_data = all_data['risky_assets']
    
    # Check for reasonable return distributions
    reasonable_stats = True
    for asset in config['assets']['risky']:
        ticker = asset['ticker']
        if ticker in risky_data.columns.levels[0]:
            returns = risky_data[ticker]['Adj Close'].pct_change().dropna()
            
            # Check for extreme outliers
            if returns.std() > 0.05:  # >5% daily vol seems unreasonable
                reasonable_stats = False
                break
            
            # Check for negative skew (crashes)
            if abs(returns.skew()) > 2:
                reasonable_stats = False
                break
    
    if reasonable_stats:
        quality_checks['Statistical Properties'].append("✅ Reasonable statistical properties")
    else:
        quality_checks['Statistical Properties'].append("⚠️ Unusual statistical properties detected")

# Print quality assessment
for category, checks in quality_checks.items():
    print(f"{category}:")
    for check in checks:
        print(f"  {check}")
    print()

print("=== DATA EXPLORATION COMPLETE ===")
print("\nNext steps:")
print("1. Review data quality issues identified above")
print("2. Run signal analysis notebook (02_signal_analysis.ipynb)")
print("3. Proceed with portfolio construction analysis")